# 融合多个模型的预测结果
通过 IOU 判断同一个 bbox

In [21]:
import pandas as pd
import os
import numpy as np
import sys

In [22]:
path = '../data/summit/'
# file_list = ['summit_frir2_027_41097', 'summit_frr101_028_29124'] #过滤bbox < 12 = merged_005 = 0.864368
file_list = ['merged_005', 'summit_frir2_027_40343'] 

In [23]:
def IOU(Reframe,GTframe):
    """
    https://blog.csdn.net/eddy_zheng/article/details/52126641
    自定义函数，计算两矩形 IOU，传入为均为矩形坐标和尺寸 [x,y, width, height]
    """
    x1, y1, width1, height1 = Reframe
    x2, y2, width2, height2 = GTframe

    endx = max(x1 + width1, x2 + width2)
    startx = min(x1, x2)
#     计算重叠部分的宽度
    width = width1 + width2 - (endx - startx)

    endy = max(y1 + height1, y2 + height2)
    starty = min(y1, y2)
#     计算重叠部分的高度
    height = height1 + height2 - (endy - starty)

    if width <=0 or height <= 0:
        ratio = 0 # 重叠率为 0 
    else:
        Area = width * height # 两矩形相交面积
        Area1 = width1 * height1
        Area2 = width2 * height2
        ratio = Area/(Area1 + Area2 - Area)
    
    return ratio,Reframe,GTframe

In [24]:
def get_coor_from_str(coor_str):
    bbox = coor_str.split('_')
    b = [int(float(bbox[0])), int(float(bbox[1])), int(float(bbox[2])), int(float(bbox[3]))]
    return b

In [25]:
def merge_bbox(bbox1, bbox2):
    bbox = []
    for i in range(len(bbox1)):
        bbox.append(int(np.round((bbox1[i] + bbox2[i]) / 2)))
    return bbox

In [26]:
def merge_coordinate(coordinate1, coordinate2):
    list1 = coordinate1.split(';')
#     print('list1 =', len(list1))
    list2 = coordinate2.split(';')
#     print('list2 =', len(list2))
#     两个连起来，两两对比
    corr_list = list1+list2
    coor_num = len(corr_list)
#     print('corr_list=', corr_list)
    merged_list = []
    result_list = []
    out_str = ''
    min_size = 14 # 12 最佳
    for i in range(0, coor_num):
        bbox1 = get_coor_from_str(corr_list[i])
#         print('%s =%s'%(i, bbox1))
        if i in merged_list:
#             print('已合并 bbox1=', bbox1)
            continue
        
        if bbox1[2] < min_size or bbox1[3] < min_size:
#             print('忽略太小的 bbox1=', bbox1)
            continue
        bbox = bbox1 #默认没有相同的，取自身 
        for j in range(i+1, coor_num):
            bbox2 = get_coor_from_str(corr_list[j])
#             print('%s =%s'%(j, bbox2))
            if j in merged_list:
#                 print('已合并 bbox2=', bbox2)
                continue
            iou = IOU(bbox1, bbox2)
            if iou[0] > 0.6:
#                 print('iou =', iou)
#                 有相同的，则合并
                bbox = merge_bbox(bbox1, bbox2)
#                 标记为已合并
                merged_list.append(j)
#                 print('bbox =', bbox)
        
        if len(result_list) > 0:
            out_str += ';'
        
        result_list.append(bbox)
        out_str += '_'.join([str(bbox[0]), str(bbox[1]), str(bbox[2]), str(bbox[3])])
#         break
#     print('merged_list =', len(merged_list))
#     print('result_list =', len(result_list))
    return out_str

In [27]:
# info_1 = pd.read_csv('%s%s.csv'%(path, file_list[0]))
# info_1.head()

In [28]:
# info_2 = pd.read_csv('%s%s.csv'%(path, file_list[1]))
# info_2.head()

In [29]:
def merge_info(info_1, info_2):
    image_num = len(info_1)
    result_df = info_1.copy()
    for i in range(0, image_num):
        info = info_1.iloc[i:i+1]
        name = info.name.values[0]
#         print('name =', name)
        coordinate = info.coordinate.values[0]
        info2 = info_2.iloc[i:i+1]
        coordinate2 = info2.coordinate.values[0]
#         print('coordinate2 =', coordinate2)
        
        result_row = result_df.iloc[i:i+1]
        if not isinstance(coordinate, str):
            coordinate_merged = coordinate2
        elif not isinstance(coordinate2, str):
            coordinate_merged = coordinate
        else:
            coordinate_merged = merge_coordinate(coordinate, coordinate2)
#         print('coordinate_merged =', coordinate_merged)
        result_row.coordinate.values[0] = coordinate_merged
        sys.stdout.write('\r>> i = %s / %s' % ((i+1),image_num))
        sys.stdout.flush()
#         if (i > 1):
#             break
    sys.stdout.write('\n')
    sys.stdout.flush()
    return result_df

# merge_info(info_1, info_2)

In [30]:
for i in range(0, len(file_list)):
    if i == 0:
        info_0 = pd.read_csv('%s%s.csv'%(path, file_list[i]))
    else:
        info_other = pd.read_csv('%s%s.csv'%(path, file_list[i]))
        info_0 = merge_info(info_0, info_other)
info_0.to_csv('%s%s.csv'%(path, 'merged'), index=False)

>> i = 5000 / 5000
